In [ ]:
#without GP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightkurve as lk
import batman
from transitleastsquares import transitleastsquares
import seaborn as sns
import os
from functools import lru_cache
from itertools import product
from scipy.signal import savgol_filter

# ------------------------------------
# 1. Cached light curve loader
# ------------------------------------
@lru_cache(maxsize=10)
def get_lightcurve(tic):
    lc_collection = lk.search_lightcurve(tic, mission="TESS", author="TESS-SPOC", cadence="long").download_all(quality_bitmask="hard")
    if lc_collection is None or len(lc_collection) == 0:
        return None
    
    lc = lc_collection.stitch().remove_nans()



    # plt.plot(lc.time.value, lc.flux.value, 'k.', markersize=1, alpha=0.5)
    # plt.show()


    return lc.to_pandas().reset_index()

# ------------------------------------
# 2. Flare masking: ±3hr from >5σ events
# ------------------------------------
def mask_flares(df):
    median = df['flux'].median()
    std = df['flux'].std()
    threshold = median + 5 * std
    flare_times = df.loc[df['flux'] > threshold, 'time'].values

    def within_3hr(t):
        return np.any(np.abs(flare_times - t) <= 0.125)

    flare_mask = df['time'].apply(within_3hr)
    return df[~flare_mask].reset_index(drop=True)


# 4. Run TLS detection for a single injection
# ------------------------------------
def run_tls(tic, time, flux):
    # cadence_days = time[1] - time[0]
    # duration_days = 2
    # wl = int(np.round(duration_days / cadence_days))
    # if wl % 2 == 0:
    #     wl += 1  # make it odd

    # print(wl)
    # flux_filter = savgol_filter(flux, wl, 2)
    # flux = flux / flux_filter

    ##LC PLOT
    # plt.plot(time, flux, 'k.', markersize=1, alpha=0.5)
    # plt.title(f"Lightcurve for {tic}, TOI {toi}")
    # plt.show()

    if len(time) == 0 or len(flux) == 0:
        return False
    tls = transitleastsquares(time, flux)
    result = tls.power(period_max = 25)

    powers = result.power
    periods = result.periods

    periodogram = pd.DataFrame({
        'period': periods,
        'power': powers
    })

    power_7_periods = periodogram[periodogram['power'] > 7]['period'].values
    print(f"TLS Periods with power > 7: {power_7_periods}")
    power_7_power = periodogram[periodogram['power'] > 7]['power'].values


    # print(f"{tic}, FAP: {result.FAP}, SDE: {result.SDE}, True Period: {true_period}")
    # # Mark as found if period matches true period or a simple alias (1/2x, 2x, 1/3x, 3x, etc.)
    # aliases = [true_period, true_period / 2, true_period * 2, true_period / 3, true_period * 3]
    # #found = result.FAP < 1e-4 and any(0.99 * alias < result.period < 1.01 * alias for alias in aliases)
    # found = any(0.99 * alias < hp < 1.01 * alias for hp in power_7_periods for alias in aliases)


    # alias_found = any(0.99 * alias < result.period < 1.01 * alias for alias in aliases)
    # print(f"Detection found: {found}")
    # if alias_found == True and found == False:
    #     print(f"True transit or alias found, but high FAP: {alias_found}")
    # #print("TLS Transit duration (hours):", result.duration * 24)


    # #TLS phase fold
    # plt.figure()
    # plt.scatter(result.folded_phase, result.folded_y, color='blue', s=10, alpha=0.5,)
    # plt.plot(result.model_folded_phase, result.model_folded_model, color='red')
    # plt.title(f"TLS Phase Folded Lightcurve for {tic}, TOI {toi}")
    # plt.xlabel('Phase')
    # plt.ylabel('Relative flux');
    # plt.show()

    #Periodogram
    # plt.figure()
    # ax = plt.gca()
    # ax.axvline(result.period, alpha=0.4, lw=3)
    # plt.xlim(np.min(result.periods), np.max(result.periods))
    # for n in range(2, 10):
    #     ax.axvline(n*result.period, alpha=0.4, lw=1, linestyle="dashed")
    #     ax.axvline(result.period / n, alpha=0.4, lw=1, linestyle="dashed")
    # plt.ylabel(r'SDE')
    # plt.xlabel('Period (days)')
    # plt.plot(result.periods, result.power, color='black', lw=0.5)
    # plt.xlim(0, max(result.periods));
    # #plt.xlim(20.8, 21.0)
    # plt.show()


    

    return result.period, result.SDE, power_7_periods, power_7_power, result.period_uncertainty, result.depth, result.rp_rs, result.FAP, result.snr

# ------------------------------------
# 5. Run detection across grid (single process)
# ------------------------------------
def run_detection_for_tic(tic):

    print(f"📥 Processing {tic}")
    lc = get_lightcurve(tic)
    if lc is None:
        print(f"❌ No light curve for {tic}")
        return pd.DataFrame()

    base_df = lc[['time', 'flux']].copy()
    base_df = mask_flares(base_df)

    time = base_df['time']
    flux = base_df['flux']
    

    # plt.plot(time, flux, 'k.', markersize=1, alpha=0.5)
    # plt.show()

    #flatten lightcurve after flare removal
    lc = lk.LightCurve(time=time, flux=flux)
    time = lc.time.value  # in days
    if len(time) < 2:
        return None

    cadence_days = time[1] - time[0]
    duration_days = 2
    wl = int(np.round(duration_days / cadence_days))
    if wl % 2 == 0:
        wl += 1  # make it odd, as required

    print(f"Cadence: {cadence_days:.5f} days, window_length: {wl}")

    lc_flat = lc.flatten(window_length=wl, polyorder=2)
    flat_df = lc_flat.to_pandas().reset_index()
    time = flat_df['time'].values
    flux = flat_df['flux'].values
    #flux = flux / np.median(flux)

    # plt.plot(time, flux, 'k.', markersize=1, alpha=0.5)
    # plt.show()


    # Get stellar parameters
    tic_id = int(tic.split()[1])
    star_params = pd.read_csv("stellar_params_CTL.csv").query(f"id == {tic_id}")
    if star_params.empty:
        print(f"⚠️ No stellar params for {tic}")
        return pd.DataFrame()

    st_radius = star_params['RAD'].values[0] #* 6.9634e8
    st_teff = star_params['teff'].values[0]
    st_tmag = star_params['tmag'].values[0]

    results = []
    tls_period, SDE_strongest,  periods_array_7, SDE_array_7, tls_period_uncertainty, tls_depth, tls_rp_rs, tls_FAP, tls_snr = run_tls(tic, time, flux)
    #detections += found

    results.append({
        'TIC': tic,
        'Stellar Radius': round(float(st_radius), 2),
        'Stellar Temperature': st_teff,
        'Stellar Magnitude': st_tmag,
        #'True Radius (Earth Radii)': round(float(true_rp_earth), 2),
        #'True Period (Days)': round(float(true_period), 4),
        #'Detection': found,
        #'Alias Detection but high FAP': alias_found,
        'TLS Period': tls_period,
        'TLS SDE strongest': SDE_strongest,
        'TLS SDE > 7 array': SDE_array_7,
        'TLS Periods array': periods_array_7,
        'TLS Period Uncertainty': tls_period_uncertainty,
        'TLS Depth': tls_depth,
        'TLS Rp/Rs': tls_rp_rs,
        'TLS FAP': tls_FAP,
        'TLS SNR': tls_snr
    })
        

    return pd.DataFrame(results)


# 7. Main driver
# ------------------------------------
if __name__ == "__main__":

    save_file = "CTL_TLS_recovery.csv"

    # Load TIC IDs from the CSV file
    star_params = pd.read_csv("stellar_params_CTL.csv")
    #star_params = star_params.iloc[530:].reset_index(drop=True)
    print(len(star_params))
    # Filter for TOI == 7033
    #star_params = star_params[star_params["id"] == 468858510]

    #star_params = star_params[928:]
    #star_params = star_params[star_params["pl_orbper"] <= 25]
    
    #star_params = star_params.iloc[40:].reset_index(drop=True)

    #star_params = star_params.iloc[[8, 12, 25, 42, 48, 51, 52, 58, 59], :] # group 1
    #star_params = star_params.iloc[[0,9,30,35,45,58], :] # group 2
    #star_params = star_params.iloc[[13,28, 31,32,33,34,36,37,39,41,43,44 ,49], :] # group 3
        # Load existing results to check for already processed TICs
    if os.path.exists(save_file):
        existing_results = pd.read_csv(save_file)
        processed_tics = set(existing_results[existing_results['Stellar Radius'].notna()]['TIC'].values)
        print(f"Found {len(processed_tics)} already processed TICs")
    else:
        processed_tics = set()
        print("No existing results file found, starting fresh")
    
    
    #for each pair of toi and tic, there's a true period and rp_earth to run

    df_save = pd.DataFrame()
for _, row in star_params.iterrows():
    #toi = row['toi']
    tic = f"TIC {int(row['id'])}"
    #true_period = row['pl_orbper']
    #true_rp_earth = row['pl_rade']
    if tic in processed_tics:
        print(f"⏭️ Skipping {tic} - already processed")
        continue
    try:
        df_results = run_detection_for_tic(tic)

        if not df_results.empty:
            df_save = pd.concat([df_save, df_results], ignore_index=True)
        else:
            df_results = pd.DataFrame([{
                'TIC': tic,
                'Stellar Radius': None,
                'Stellar Temperature': None,
                'Stellar Magnitude': None,
                'TLS Period': None,
                'TLS SDE strongest': None,
                'TLS SDE > 7 array': None,
                'TLS Periods array': None,
                'TLS Period Uncertainty': None,
                'TLS Depth': None,
                'TLS Rp/Rs': None,
                'TLS FAP': None,
                'TLS SNR': None
            }])
        
            df_save = pd.concat([df_save, df_results], ignore_index=True)

        # ✅ Save result
        if os.path.exists(save_file):
            existing = pd.read_csv(save_file)
            existing = existing[existing['TIC'] != tic]
            updated = pd.concat([existing, df_results], ignore_index=True)
        else:
            updated = df_results
        updated.to_csv(save_file, index=False)

    except Exception as e:
        print(f"❌ Error occurred for {tic}: {e}")
        df_results = pd.DataFrame([{
                'TIC': tic,
                'Stellar Radius': None,
                'Stellar Temperature': None,
                'Stellar Magnitude': None,
                'TLS Period': None,
                'TLS SDE strongest': None,
                'TLS SDE > 7 array': None,
                'TLS Periods array': None,
                'TLS Period Uncertainty': None,
                'TLS Depth': None,
                'TLS Rp/Rs': None,
                'TLS FAP': None,
                'TLS SNR': None
            }])
        
        df_save = pd.concat([df_save, df_results], ignore_index=True)

        # ✅ Save even on error
        if os.path.exists(save_file):
            existing = pd.read_csv(save_file)
            existing = existing[existing['TIC'] != tic]
            updated = pd.concat([existing, df_results], ignore_index=True)
        else:
            updated = df_results
        updated.to_csv(save_file, index=False)

11910
No existing results file found, starting fresh
📥 Processing TIC 288314668


Cadence: 0.02083 days, window_length: 97
Transit Least Squares TLS 1.0.31 (22 Nov 2021)
Creating model cache for 42 durations
Searching 27336 data points, 106272 periods from 0.602 to 24.998 days
Using all 10 CPU threads


100%|██████████| 106272/106272 periods | 09:40<00:00


Searching for best T0 for period 15.73704 days


Process SpawnPoolWorker-46230:
Traceback (most recent call last):
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
_pickle.UnpicklingError: invalid load key, '\x00'.
Process SpawnPoolWorker-46231:
Traceback (most recent call last):
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/multiprocessing/process

...

/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/site-packages/transitleastsquares/main.py:396: RuntimeWarning: divide by zero encountered in double_scalars
  odd_even_mismatch = odd_even_difference / odd_even_std_sum
/Users/danayaptangco/opt/anaconda3/envs/Mulders/lib/python3.9/site-packages/transitleastsquares/main.py:411: UserWarning:

TLS Periods with power > 7: [ 8.88913072  8.89589208 10.95610639 21.91215542]
📥 Processing TIC 207339173
Cadence: 0.02083 days, window_length: 97
Transit Least Squares TLS 1.0.31 (22 Nov 2021)
Creating model cache for 42 durations
Searching 13605 data points, 108870 periods from 0.602 to 24.999 days
Using all 10 CPU threads


100%|██████████| 108870/108870 periods | 06:48<00:00


Searching for best T0 for period 5.38375 days



  2%|▏         | 279/11728 [00:03<02:39, 71.74it/s]